<!-- Logo al 25% -->
<td width="45%" align="left" valign="middle">
  <img src="https://www.upc.edu/comunicacio/ca/identitat/descarrega-arxius-grafics/fitxers-marca-principal/upc-positiu-p3005.png" width="300">
</td>

<!-- Texto al 75%, alineado a la derecha -->
<td width="5%" align="right" valign="middle">
  <p style="margin: 0;"><b>Intelligence Data Science and Artificial Intelligence (IDEAI)</b></p>
  <p style="margin: 0;"><b>Grau en Estadística (UB - UPC)</b></p>
  <p style="margin: 0;">Mètodes Estadístics per la Mineria de Dades (MeMDa)</p>
</td>

# 🤖 **Conversaciones con IBAI**

In [1]:
import os
import time
import datetime
import numpy as np
import random
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

In [2]:
if torch.cuda.is_available():
    print("Usar GPU")
    device = torch.device("cuda")
    batch_size = 3
else:
    print("usar CPU")
    device = torch.device("cpu")
    batch_size = 1

Usar GPU


In [3]:
# Load the GPT tokenizer.
tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained("flax-community/gpt-2-spanish")

control_code = "ibai"

special_tokens_dict = {
         "additional_special_tokens": ['f"<|{control_code}|>"'],
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
unk_tok_emb = model.transformer.wte.weight.data[tokenizer.unk_token_id, :]
for i in range(num_added_toks):
        model.transformer.wte.weight.data[-(i+1), :] = unk_tok_emb

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [4]:
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import requests
import os
import time

class GPT2Dataset(Dataset):
    def __init__(self, control_code, tokenizer, archivo_texto, max_length=768,
                 max_retries=3, timeout=15):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        print("loading text...")

        raw_text = None

        # 1) Detectar si es URL o archivo local
        if archivo_texto.startswith("http://") or archivo_texto.startswith("https://"):
            print(f"Descargando texto desde URL: {archivo_texto}")

            last_error = None
            for attempt in range(1, max_retries + 1):
                try:
                    resp = requests.get(archivo_texto, timeout=timeout)
                    resp.raise_for_status()
                    raw_text = resp.text
                    print(f"Descarga completada en el intento {attempt}.")
                    break
                except requests.exceptions.RequestException as e:
                    last_error = e
                    print(f"Intento {attempt}/{max_retries} fallido: {e}")
                    if attempt < max_retries:
                        # Espera incremental: 2s, 4s, 6s...
                        sleep_sec = 2 * attempt
                        print(f"Reintentando en {sleep_sec} segundos...")
                        time.sleep(sleep_sec)

            if raw_text is None:
                raise ConnectionError(
                    f"No se pudo descargar el texto desde la URL tras {max_retries} intentos.\n"
                    f"Último error: {last_error}"
                )

        else:
            # 2) Archivo local
            if not os.path.exists(archivo_texto):
                raise FileNotFoundError(f"Archivo no encontrado: {archivo_texto}")
            print(f"Leyendo archivo local: {archivo_texto}")
            with open(archivo_texto, "r", encoding="utf-8") as f:
                raw_text = f.read()

        # 3) Normalizar texto y dividir en líneas
        raw_text = raw_text.lower()
        sentences = raw_text.split("\n")
        print("qty:", len(sentences))

        # 4) Tokenizar línea por línea
        for row in tqdm(sentences):
            if not row.strip():
                continue  # saltar líneas vacías

            text = f"<|startoftext|><|{control_code}|>{row}<|endoftext|>"
            encodings_dict = tokenizer(
                text,
                truncation=True,
                max_length=max_length,
                padding="max_length"
            )
            self.input_ids.append(torch.tensor(encodings_dict["input_ids"]))
            self.attn_masks.append(torch.tensor(encodings_dict["attention_mask"]))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [5]:

dataset = GPT2Dataset(control_code, tokenizer, archivo_texto="https://raw.githubusercontent.com/jbagnato/machine-learning/master/gpt-2/ibai_textos.txt", max_length=768)
# Split into training and validation sets
train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


loading text...
Descargando texto desde URL: https://raw.githubusercontent.com/jbagnato/machine-learning/master/gpt-2/ibai_textos.txt
Descarga completada en el intento 1.
qty: 10293


100%|██████████| 10293/10293 [00:07<00:00, 1291.13it/s]

10,189 training samples
  103 validation samples


In [6]:
epochs = 1
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 50  # print a sample every 50 batches
optimizer = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_steps, num_training_steps = total_steps)
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [7]:
total_t0 = time.time()
model = model.to(device)

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()
        outputs = model(  b_input_ids, labels=b_labels,
                          attention_mask = b_masks, token_type_ids=None )
        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss
        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))
        loss.backward()
        optimizer.step()
        scheduler.step()
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    t0 = time.time()
    total_eval_loss = 0
    nb_eval_steps = 0
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 1 ========
Training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


  Batch    50  of  3,397. Loss: 10.290851593017578.   Elapsed: 0:00:35.
  Batch   100  of  3,397. Loss: 6.515345096588135.   Elapsed: 0:01:12.
  Batch   150  of  3,397. Loss: 0.2992258369922638.   Elapsed: 0:01:49.
  Batch   200  of  3,397. Loss: 0.19885745644569397.   Elapsed: 0:02:25.
  Batch   250  of  3,397. Loss: 1.587853193283081.   Elapsed: 0:03:02.
  Batch   300  of  3,397. Loss: 0.23721644282341003.   Elapsed: 0:03:39.
  Batch   350  of  3,397. Loss: 0.28201645612716675.   Elapsed: 0:04:16.
  Batch   400  of  3,397. Loss: 0.3308331370353699.   Elapsed: 0:04:52.
  Batch   450  of  3,397. Loss: 0.2574785053730011.   Elapsed: 0:05:29.
  Batch   500  of  3,397. Loss: 0.22272807359695435.   Elapsed: 0:06:06.
  Batch   550  of  3,397. Loss: 0.27064669132232666.   Elapsed: 0:06:43.
  Batch   600  of  3,397. Loss: 0.2688136100769043.   Elapsed: 0:07:19.
  Batch   650  of  3,397. Loss: 0.18267841637134552.   Elapsed: 0:07:56.
  Batch   700  of  3,397. Loss: 0.2957932651042938.   Elapse

In [8]:
output_dir = './model_gpt_ibai/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./model_gpt_ibai/tokenizer_config.json',
 './model_gpt_ibai/special_tokens_map.json',
 './model_gpt_ibai/vocab.json',
 './model_gpt_ibai/merges.txt',
 './model_gpt_ibai/added_tokens.json',
 './model_gpt_ibai/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Directorio donde guardaste el modelo
input_dir = './model_gpt_ibai/'
# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(input_dir)
# Cargar el modelo
model = AutoModelForCausalLM.from_pretrained(input_dir)

In [10]:
model.eval()
prompt = "<|startoftext|>" + "<|ibai|>" + "¿ opinas de los alumnos que estudias mineria de datos ?"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                                generated,
                                num_return_sequences=3,
                                max_length = 300,
                                do_sample=True,
                                top_k=50,
                                top_p=0.95,
                                pad_token_id=tokenizer.eos_token_id
                                )
for i, sample_output in enumerate(sample_outputs):
    print(f"{i}: {tokenizer.decode(sample_output, skip_special_tokens=True)}\n")

0: <|ibai|>¿ opinas de los alumnos que estudias mineria de datos ? ¡sí! sí, sí.  estoy aprendiendo a tocar, tío.  estoy aprendiendo, tío.  tengo la carrera en el lado derecho. 

1: <|ibai|>¿ opinas de los alumnos que estudias mineria de datos ? porque dices que tienes conocimientos, ¿ no ? creo que no te conoces, ni siquiera sabes tu historia.  yo creo que si tienes esos conocimientos yo creo que eres un persona de verdad.  y por eso te entiendo.  y creo que de verdad me he pasado mucho tiempo, pero no me he sentido insultado, no sé si eso es bueno o malo. 

2: <|ibai|>¿ opinas de los alumnos que estudias mineria de datos ? ¿ vas de un curso de educación primaria a un curso de educación secundaria ? a mí me parece bastante gracioso, es una experiencia algo original y enriquecedora.  pero vamos, el punto es que al final es donde tienes que estar y no solo cuando estás en el proceso.  a mí me ha pasado de no me acuerdo, es decir, siempre te quedas un poco sin saber si estudiar o estudiar

In [14]:
# Instalar dependencias
!pip install transformers torch ipywidgets --quiet

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# -----------------------------
# 1️⃣ Cargar modelo y tokenizer
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_dir = './model_gpt_ibai/'

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir)
model.to(device)
model.eval()

# -----------------------------
# 2️⃣ Función para generar respuesta
# -----------------------------
# def generar_respuesta(prompt, max_length=200):
#    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
#    output_ids = model.generate(
#        input_ids,
#        max_length=max_length,
#        do_sample=True,
#        top_k=50,
#        top_p=0.95,
#        num_return_sequences=1,
#        pad_token_id=tokenizer.eos_token_id
#    )
#    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def generar_respuesta(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + 100,  # generar solo hasta 100 tokens más
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )
    # Decodificar
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Solo devolver la parte que el modelo añadió después del prompt
    response = output_text[len(prompt):].strip()
    return response

# -----------------------------
# 3️⃣ Crear interfaz estilo ChatGPT
# -----------------------------
chat_history = []

# Output container donde se mostrará el chat
chat_output = widgets.Output(layout={'border': '1px solid #ccc', 'height': '400px', 'overflow_y': 'auto', 'padding': '10px'})

# Caja de texto para ingresar mensajes
input_box = widgets.Text(
    placeholder='Escribe tu mensaje aquí...',
    layout=widgets.Layout(width='90%')
)

# Botón para enviar
send_button = widgets.Button(description="Enviar", button_style='primary')

# Función para actualizar el chat con burbujas
def mostrar_chat():
    with chat_output:
        clear_output(wait=True)
        for sender, message in chat_history:
            if sender == "Tú":
                display(HTML(f"<div style='text-align:right; background-color:#DCF8C6; padding:8px; margin:5px; border-radius:10px; max-width:70%; float:right; clear:both'>{message}</div>"))
            else:
                display(HTML(f"<div style='text-align:left; background-color:#F1F0F0; padding:8px; margin:5px; border-radius:10px; max-width:70%; float:left; clear:both'>{message}</div>"))
        display(HTML("<div style='clear:both'></div>"))

# Función que se ejecuta al enviar mensaje
def enviar_mensaje(_):
    global chat_history
    user_message = input_box.value.strip()
    if not user_message:
        return
    input_box.value = ""
    chat_history.append(("Tú", user_message))

    # Construir prompt para el modelo con todo el historial
    prompt = "<|startoftext|><|ibai|>\n"
    for sender, msg in chat_history:
        if sender == "Tú":
            prompt += f"Tú: {msg}\n"
        else:
            prompt += f"Ibai: {msg}\n"
    prompt += "Ibai: "

    mostrar_chat()  # mostrar mensaje del usuario

    # Generar respuesta
    response = generar_respuesta(prompt)
    chat_history.append(("Ibai", response))
    mostrar_chat()

# Conectar botón y tecla Enter
send_button.on_click(enviar_mensaje)
input_box.on_submit(enviar_mensaje)

# Mostrar la interfaz
display(chat_output, widgets.HBox([input_box, send_button]))


The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


Output(layout=Layout(border='1px solid #ccc', height='400px', overflow_y='auto', padding='10px'))